- В папку для обработки положить файлы которые надо обработать
- Проверяет на наличие найденных совпадений в справочниках
- Проверять чтобы было соответствие колонок у всех файлов


In [30]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime

In [53]:
path2 = r"D:\Analysis Burn\OSH\Для обработки"
xl_files = glob.glob(os.path.join(path2, "*.xlsx"))

In [54]:
# frame2.to_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv", index=False)

In [55]:
li2 = []

for f2 in xl_files:
      
    # read the csv file
    df2 = pd.read_excel(f2)
    df2 = df2.drop(df2.index[-1])
    
    li2.append(df2)  
    
    
    # print the location and filename
    # print('Location:', f)
    # print('File Name:', f.split("\\")[-1])
      
    # print the content
    # print('Content:')
    # display(df)
    # print()

frame2 = pd.concat(li2, axis=0, ignore_index=True)

In [3]:
# frame2 = pd.read_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv")

In [56]:
frame2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69367 entries, 0 to 69366
Data columns (total 16 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Основной менеджер                          69262 non-null  object 
 1   Родитель                                   68011 non-null  object 
 2   Бизнес-регион                              68011 non-null  object 
 3   Группа клиентов (Клиенты)                  25052 non-null  object 
 4   Клиент                                     69367 non-null  object 
 5   Код                                        69367 non-null  object 
 6   Адрес доставки                             69367 non-null  object 
 7   Заказ клиента / Реализация.Дата отгрузки   68790 non-null  object 
 8   Бренд (Справочник "Номенклатура" (Общие))  69367 non-null  object 
 9   Группа                                     69367 non-null  object 
 10  Номенклатура          

In [57]:
frame2['Основной менеджер'].unique()

array(['KAM 001 (Шулагин)', 'PRE МОПТ MSK', 'PRE MSK 005', 'PRE MSK 010',
       'REG MAN 001 (Такаев: ЮФО, СНГ)', 'PRE MSK 011', 'PRE MSK 026',
       'REG MAN 007 (Меркурьев)', 'PRE MSK 001', 'PRE MSK 004',
       'PRE MSK 007', 'PRE MSK 022', 'PRE MSK 015',
       'REG MAN 005 (Цыганков: ДВФО, ПФО)', 'PRE MSK 024',
       'PRE MO ZELENOGRAD 001', 'PRE MSK 002', 'PRE MSK 021',
       'PRE MSK 003', nan, 'PRE MSK 017', 'PRE MSK 025', 'PRE MSK 016',
       'PRE MSK 006', 'PRE MSK 009', 'PRE MSK 013', 'PRE MSK 023',
       'Telesales 002 (Юшкин)', 'PRE MO ELEKTROSTAL 001',
       'PRE MSK 018 (MO)', 'Telesales 001 (Маслов)', 'PRE MO KOLOMNA 001',
       'PRE MSK 008 (MO)', 'PRE MSK 014', 'PRE MSK 020', 'PRE MSK 027',
       'Telesales 003 (Творогов)', 'PRE MO NARO-FOMINSK 001',
       'PRE MO SERPUHOV 001', 'PRE MSK 012', 'PRE MO POSAD 001',
       'PRE MSK 019', 'Абильфазов Азим', 'Дудырев Владимир', 'Фаткин Ким',
       'Асланян Ашхен', 'Баранов Алексей', 'Сафронов Николай'],
      dt

In [58]:
frame2['СегментПартнера'] = frame2['СегментПартнера'].fillna("Не указано")

In [59]:
frame2['СегментПартнера'] = frame2['СегментПартнера'].str.replace('Кафе/Ресторан', 'HoReCa', regex=False)

In [60]:
# frame2['Основной менеджер'] = frame2['Основной менеджер'].str.replace(' (МО)', '', regex=False)

In [61]:
frame2['Граммовка'] = ( frame2['Вес (нетто), кг'] / frame2['Количество'] )* 1000 

In [62]:
frame2['цена'] = frame2['Выручка'] / frame2['Количество']

In [63]:
frame2['Бренд (Справочник "Номенклатура" (Общие))'] =  frame2['Бренд (Справочник "Номенклатура" (Общие))'].str.upper()

frame2['Номенклатура'] =  frame2['Номенклатура'].str.upper()

frame2['Группа'] =  frame2['Группа'].str.upper()

In [64]:
frame2['Бренд (Справочник "Номенклатура" (Общие))'] =  frame2['Бренд (Справочник "Номенклатура" (Общие))'].str.strip()

frame2['Номенклатура'] =  frame2['Номенклатура'].str.strip()

frame2['Группа'] =  frame2['Группа'].str.strip()

In [65]:
brands_dim = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name = "варианты брендов")
groups_dim = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name = "варианты групп")
delicious_dim = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name = "варианты вкусов")

In [66]:
frame2 = frame2.merge(
    brands_dim,
    how = 'left',
    left_on='Группа', 
    right_on='Бренд (исходный)',
    suffixes=('_left', '_right'),
)

In [67]:
frame2['Бренд'].isna().unique()

array([False])

In [68]:
frame2['Бренд+Вкус'] = frame2['Бренд'].str.upper() + frame2['Номенклатура']

In [69]:
frame2 = frame2.merge(
    groups_dim,
    how = 'left',
    left_on='Группа', 
    right_on='Исходная группа',
    suffixes=('_left', '_right'),
)

In [70]:
frame2['Группа_right'].isna().unique()

array([False])

In [71]:
frame2 = frame2.merge(
    delicious_dim,
    how = 'left',
    left_on='Бренд+Вкус', 
    right_on='Исходный вкус',
    suffixes=('_left', '_right')
)

In [72]:
frame2['Вкус'].isna().unique()

array([False])

In [50]:
# a = frame2.loc[frame2['Вкус'].isna()]

In [52]:
# a['Бренд+Вкус'].unique()

array(['BANGERBANGER GRAPEFRUIT 100ГР МРК',
       'BANGERBANGER SAINT-TROPEZ 100ГР МРК',
       'BLACKBURNBURN BLACK ICEBERG 25ГР МРК',
       'BANGERBANGER GRAPEFRUIT 25ГР МРК',
       'BANGERBANGER SAINT-TROPEZ 25ГР МРК',
       'BANGERBANGER LEMON TONIC 100ГР МРК'], dtype=object)

In [73]:
frame2 = frame2.drop(
    columns = {
        'Группа.1', 'Бренд (Справочник "Номенклатура" (Общие))', 'Номенклатура', 'Бренд (исходный)', 'Бренд+Вкус',
        'Группа_left', 'Исходная группа', 'Brand', 'Граммовка_right', 'Исходный вкус', 'Код_right', 
    }
)

In [74]:
frame2 = frame2.rename(
    columns={
      'Код_left': "Код",
       'Граммовка_left': "Граммовка",
        "Группа_right": "Группа"
        
    }
)

In [121]:
# frame.to_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv", index=False)

In [120]:
# frame = frame.fillna('29.05.2023')

In [119]:
frame['Заказ клиента / Реализация.Дата отгрузки'].unique()

array(['10.08.2022', '13.08.2022', '26.08.2022', '09.08.2022',
       '31.08.2022', '02.08.2022', '05.08.2022', '17.08.2022',
       '03.08.2022', '04.08.2022', '27.08.2022', '23.08.2022',
       '11.08.2022', '12.08.2022', '24.08.2022', '20.08.2022',
       '01.08.2022', '08.08.2022', '29.08.2022', '30.08.2022',
       '22.08.2022', '06.08.2022', '18.08.2022', '19.08.2022',
       '25.08.2022', '16.08.2022', '15.08.2022', '27.04.2022',
       '26.04.2022', '13.04.2022', '06.04.2022', '21.04.2022',
       '29.04.2022', '12.04.2022', '28.04.2022', '01.05.2022',
       '25.04.2022', '01.04.2022', '16.04.2022', '07.04.2022',
       '20.04.2022', '19.04.2022', '04.04.2022', '22.04.2022',
       '11.04.2022', '30.04.2022', '18.04.2022', '02.04.2022',
       '23.04.2022', '14.04.2022', '09.04.2022', '08.04.2022',
       '05.04.2022', '15.04.2022', '10.12.2022', '09.12.2022',
       '21.12.2022', '29.12.2022', '08.12.2022', '24.12.2022',
       '30.12.2022', '15.12.2022', '22.12.2022', '28.12

In [79]:
# frame = pd.concat([frame3, frame2])

In [149]:
frame['СегментПартнера'] = frame['СегментПартнера'].str.lower()

## ABC

In [93]:
# frame = pd.read_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv")

In [95]:
frame['month'] = pd.to_datetime(frame['Заказ клиента / Реализация.Дата отгрузки'], dayfirst = True)

In [97]:
frame['month'] = frame['month'].dt.month

In [69]:
# abc_temp = frame.groupby(
#     ['СегментПартнера','Код', 'month']
# ).agg({"Вес (нетто), кг":"sum"}).reset_index().sort_values("Вес (нетто), кг",ascending = False)

In [110]:
abc_temp = frame.pivot_table(
    values='Вес (нетто), кг',
    index=['СегментПартнера','Код'],
    columns=['month']
).reset_index()

In [111]:
# здесь править
abc_temp['Ср_в_месяц'] = abc_temp.iloc[:, -12:].median(axis=1)

In [112]:
abc_temp

month,СегментПартнера,Код,1,2,3,4,5,6,7,8,9,10,11,12,Ср_в_месяц
0,horeca,00-00000740,0.192308,0.307692,0.200000,0.285714,0.347826,0.2,0.333333,0.22,0.28,0.244444,0.266667,0.213333,0.255556
1,horeca,00-00001042,0.200000,NaN,NaN,0.200000,0.200000,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,0.200000
2,horeca,00-00001188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,0.200000
3,horeca,00-00001254,0.100000,NaN,0.111111,NaN,0.133333,NaN,0.100000,NaN,NaN,0.100000,NaN,NaN,0.100000
4,horeca,00-00001401,0.070000,0.054000,NaN,NaN,NaN,NaN,0.145833,NaN,NaN,NaN,NaN,NaN,0.070000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5264,табаконист,УТ-УТ000215,NaN,NaN,NaN,NaN,0.025000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025000
5265,табаконист,УТ-УТ000219,NaN,NaN,NaN,NaN,0.025000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025000
5266,табаконист,УТ-УТ000222,NaN,NaN,NaN,NaN,0.028788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028788
5267,табаконист,УТ-УТ000224,NaN,NaN,NaN,NaN,0.025000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025000


In [114]:
abc_temp = abc_temp.drop(columns={1,2,3,4,5,6,7,8,9,10,11,12})

In [116]:
abc = abc_temp.sort_values("Ср_в_месяц",ascending = False)

In [152]:
abc = abc.rename(columns = {'Вес (нетто), кг': 'Сумма продаж'})

In [119]:
# Разделение данных по сегментам
segment_groups = abc.groupby('СегментПартнера')

In [120]:
segment_groups.head()

month,СегментПартнера,Код,Ср_в_месяц
2662,не указано,УТ-00016592,5.000000
3446,спец. кальянный магазин,УТ-00012816,3.692308
1463,личные нужды,00-00008426,3.533333
1488,мопт,00-00000354,2.813658
1527,мопт,УТ-00011707,2.591216
...,...,...,...
3100,продуктовый магазин,УТ-00016546,0.075000
3097,продуктовый магазин,УТ-00016134,0.075000
3078,пивной магазин,УТ-00016488,0.036667
3069,пивной магазин,УТ-00012905,0.035568


In [121]:
def perform_abc_analysis(segment_df):
    # Сортировка клиентов по сумме продаж в порядке убывания
    sorted_df = segment_df.sort_values(by='Ср_в_месяц', ascending=False)

    # Расчет общей суммы продаж
    total_sales = segment_df['Ср_в_месяц'].sum()

    # Расчет кумулятивной суммы продаж и доли кумулятивной суммы
    sorted_df['Кумулятивная сумма продаж'] = sorted_df['Ср_в_месяц'].cumsum()
    sorted_df['Доля кумулятивной суммы'] = sorted_df['Кумулятивная сумма продаж'] / total_sales

    # Определение классов клиентов на основе доли кумулятивной суммы
    sorted_df['Класс'] = pd.cut(sorted_df['Доля кумулятивной суммы'], bins=[0, 0.8, 0.95, 1.01], labels=['A', 'B', 'C'])

    return sorted_df

In [122]:
# Применение ABC-анализа к каждому сегменту
abc_results = segment_groups.apply(perform_abc_analysis)

In [126]:
abc_results.head()

month               СегментПартнера          Код  Ср_в_месяц  \
СегментПартнера                                                
horeca          88           horeca  УТ-00014869    1.507692   
                40           horeca  УТ-00012967    0.733333   
                16           horeca  00-00008569    0.531579   
                89           horeca  УТ-00014902    0.506061   
                226          horeca  УТ-УТ000171    0.497222   

month                Кумулятивная сумма продаж  Доля кумулятивной суммы Класс  
СегментПартнера                                                                
horeca          88                    1.507692                 0.032726     A  
                40                    2.241026                 0.048644     A  
                16                    2.772605                 0.060182     A  
                89                    3.278665                 0.071167     A  
                226                   3.775887                 0.081960     A

In [158]:
# abc_results.to_excel(r"D:\Analysis Burn\OSH\ABC_Clients.xlsx", index=False)

In [127]:
abc_results.to_csv(r"D:\Analysis Burn\OSH\ABC_Clients.csv", index=False)

In [128]:
abc_results['СегментПартнера+Код'] = abc_results['СегментПартнера'] + abc_results['Код']

In [129]:
abc_results['СегментПартнера+Код'].is_unique

True

In [281]:
abc_results

month                СегментПартнера          Код  Ср_в_месяц  \
СегментПартнера                                                 
horeca          88            horeca  УТ-00014869    1.507692   
                40            horeca  УТ-00012967    0.733333   
                16            horeca  00-00008569    0.531579   
                89            horeca  УТ-00014902    0.506061   
                226           horeca  УТ-УТ000171    0.497222   
...                              ...          ...         ...   
табаконист      4068      табаконист  УТ-00012454    0.024722   
                4083      табаконист  УТ-00012665    0.024615   
                3872      табаконист  00-00010088    0.024500   
                4161      табаконист  УТ-00013374    0.024464   
                4210      табаконист  УТ-00013719    0.024444   

month                 Кумулятивная сумма продаж  Доля кумулятивной суммы  \
СегментПартнера                                                            
horeca          88                     1.507692                 0.032726   
                40                     2.241026                 0.048644   
                16                     2.772605                 0.060182   
                89                     3.278665                 0.071167   
                226                    3.775887                 0.081960   
...                                         ...                      ...   
табаконист      4068                  79.273026                 0.998765   
                4083                  79.297641                 0.999075   
                3872                  79.322141                 0.999384   
                4161                  79.346606                 0.999692   
                4210                  79.371050                 1.000000   

month                Класс    СегментПартнера+Код  
СегментПартнера                                    
horeca          88       A      horecaУТ-00014869  
                40       A      horecaУТ-00012967  
                16       A      horeca00-00008569  
                89       A      horecaУТ-00014902  
                226      A      horecaУТ-УТ000171  
...                    ...                    ...  
табаконист      4068     C  табаконистУТ-00012454  
                4083     C  табаконистУТ-00012665  
                3872     C  табаконист00-00010088  
                4161     C  табаконистУТ-00013374  
                4210     C  табаконистУТ-00013719  

[5269 rows x 7 columns]

In [130]:
frame['СегментПартнера+Код'] = frame['СегментПартнера'] + frame['Код']

In [131]:
frame2 = frame.merge(abc_results[['Класс', 'СегментПартнера+Код']], how='left', on="СегментПартнера+Код", suffixes=('_left', '_right'))

In [134]:
frame2.to_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv")

## Потерянные клиенты

In [192]:
frame2 = pd.read_csv(r"D:\Analysis Burn\OSH\Всё в сборе.csv")

In [397]:
frame2['Класс'].unique()

array(['A', 'B', 'C'], dtype=object)

In [32]:
# # преобразование колонки в дата подобный тип
# def convert_to_date(date_str):
#     date_obj = datetime.strptime(date_str, '%d.%m.%Y').date()
#     return date_obj

# frame2['Заказ клиента / Реализация.Дата отгрузки'] = frame2['Заказ клиента / Реализация.Дата отгрузки'].apply(convert_to_date)

In [202]:
frame2['номер_месяца'] = pd.to_datetime(frame2['Заказ клиента / Реализация.Дата отгрузки'], dayfirst=True)
frame2['номер_месяца'] = frame2['номер_месяца'].dt.month

frame2['название_месяца'] = pd.to_datetime(frame2['Заказ клиента / Реализация.Дата отгрузки'], dayfirst=True)
frame2['название_месяца'] = frame2['название_месяца'].dt.month_name()

frame2['год'] = pd.to_datetime(frame2['Заказ клиента / Реализация.Дата отгрузки'], dayfirst=True)
frame2['год'] = frame2['год'].dt.year

frame2['год_месяц'] = frame2['год'].astype(str) + '_' + frame2['номер_месяца'].astype(str)

In [429]:
pivot_table = frame2.pivot_table(
    values='Вес (нетто), кг',
    index=['Код','Клиент','СегментПартнера'],
    columns='год_месяц',
    aggfunc=lambda x: 1 if sum(x) > 0 else 0,
    fill_value=0
).reset_index()

In [431]:
# сумма 3-х месяцев
pivot_table['закуп_3_месяца'] = pivot_table.iloc[:, -3:].sum(axis=1)
pivot_table['закуп_4_месяца_назад'] = pivot_table.iloc[:, -5:-4].sum(axis=1)

In [433]:
pivot_table['потерянный_клиент'] = 0
pivot_table.loc[(pivot_table['закуп_4_месяца_назад'] == 1) & (pivot_table['закуп_3_месяца'] == 0),'потерянный_клиент'] = 1

In [436]:
pivot_table = pivot_table.loc[ pivot_table['потерянный_клиент'] == 1 ]

In [438]:
pivot_table=pivot_table[['Клиент','СегментПартнера','Код']]

In [439]:
pivot_table['СегментПартнера+Код'] = pivot_table['СегментПартнера'] + pivot_table['Код']

C:\Users\User\AppData\Local\Temp\ipykernel_8376\23198054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_table['СегментПартнера+Код'] = pivot_table['СегментПартнера'] + pivot_table['Код']


In [442]:
lost = pivot_table.merge(abc_results[['Класс', 'СегментПартнера+Код', 'Ср_в_месяц']], how='left', on="СегментПартнера+Код", suffixes=('_left', '_right'))

In [445]:
lost

,Клиент,СегментПартнера,Класс,Ср_в_месяц
0,Gangster Bar (Страстной),бар/ночной клуб,A,0.200000
1,МЯТА (Велозаводская),кальянная,B,0.175000
2,МЯТА (ПЕТРОВКА),кальянная,C,0.100000
3,Ахматовская Александра Юрьевна,спец. кальянный магазин,A,0.108333
4,Алексей Серенков Алексеевич,не указано,A,0.200000
...,...,...,...,...
239,Сергей,не указано,B,0.100000
240,Серафима Александровна Пырсова,не указано,C,0.033333
241,Арсений Торговцев,не указано,C,0.025000
242,кудрявцев марк,не указано,C,0.029167


In [446]:
lost = lost.drop(columns={'Код', 'СегментПартнера+Код'})

KeyError: "['СегментПартнера+Код', 'Код'] not found in axis"

In [447]:
lost.to_excel(r"D:\Analysis Burn\OSH\lost_clients.xlsx", index=False)